<a href="https://colab.research.google.com/github/mmarin11/Colabfiles/blob/main/Practica2/Practica2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import re
from datetime import datetime
from collections import defaultdict
import statistics

def extraer_datos_acciones(ruta_archivo):
    """
    Extrae los datos de acciones del archivo especificado.
    """
    datos = []
    contenido = ""

    # Intentar diferentes codificaciones
    codificaciones = ['utf-8', 'latin-1']

    for codificacion in codificaciones:
        try:
            with open(ruta_archivo, 'r', encoding=codificacion) as archivo:
                contenido = archivo.read()
            print(f"✅ Archivo leído exitosamente con codificación: {codificacion}")
            break
        except UnicodeDecodeError:
            continue
        except FileNotFoundError:
            print(f"❌ Error: No se encontró el archivo '{ruta_archivo}'")
            return []
        except Exception as e:
            print(f"❌ Error al leer el archivo: {e}")
            return []
    else:
        print("❌ No se pudo leer el archivo con ninguna codificación.")
        return []

    if not contenido:
        print("❌ El archivo está vacío.")
        return []

    # Patrón para extraer información de cada informe
    # Incluir variaciones posibles de "Acción" con diferentes codificaciones
    patrones = [
        r'Asunto: Informe de Acci[oóÛû]n - ([A-Z]+).*?Fecha: ([0-9-]+\s[0-9:]+).*?Valor actual: \$([0-9,.]+)',
        r'Asunto: Informe de AcciÛn - ([A-Z]+).*?Fecha: ([0-9-]+\s[0-9:]+).*?Valor actual: \$([0-9,.]+)',
        r'Asunto: Informe de Acción - ([A-Z]+).*?Fecha: ([0-9-]+\s[0-9:]+).*?Valor actual: \$([0-9,.]+)'
    ]

    coincidencias = []
    for patron in patrones:
        coincidencias = re.findall(patron, contenido, re.DOTALL)
        if coincidencias:
            print(f"✅ Datos extraídos con el patrón: {patron[:50]}...")
            break

    if not coincidencias:
        print("❌ No se encontraron datos en el formato esperado.")
        print("Primeras 500 caracteres del archivo:")
        print(repr(contenido[:500]))
        return []

    for simbolo, fecha_str, valor_str in coincidencias:
        try:
            fecha = datetime.strptime(fecha_str, '%Y-%m-%d %H:%M:%S')
            valor = float(valor_str.replace(',', ''))

            datos.append({
                'simbolo': simbolo,
                'fecha': fecha,
                'valor': valor,
                'fecha_str': fecha_str
            })
        except ValueError as e:
            print(f"⚠️ Error al procesar registro: {e}")
            continue

    print(f"✅ Se extrajeron {len(datos)} registros válidos.")
    return datos

def agrupar_por_empresa(datos):
    """
    Agrupa los datos por empresa.
    """
    empresas = defaultdict(list)
    for dato in datos:
        empresas[dato['simbolo']].append(dato)

    # Ordenar cada empresa por fecha
    for simbolo in empresas:
        empresas[simbolo].sort(key=lambda x: x['fecha'])

    return empresas

def calcular_estadisticas(valores):
    """
    Calcula estadísticas básicas para una lista de valores.
    """
    if not valores:
        return {}

    return {
        'cantidad': len(valores),
        'minimo': min(valores),
        'maximo': max(valores),
        'promedio': statistics.mean(valores),
        'mediana': statistics.median(valores),
        'desviacion': statistics.stdev(valores) if len(valores) > 1 else 0
    }

def analizar_empresa(simbolo, registros):
    """
    Analiza los datos de una empresa específica.
    """
    valores = [r['valor'] for r in registros]
    stats = calcular_estadisticas(valores)

    # Información adicional
    primer_valor = registros[0]['valor']
    ultimo_valor = registros[-1]['valor']
    variacion = ultimo_valor - primer_valor
    variacion_porcentual = (variacion / primer_valor) * 100 if primer_valor != 0 else 0

    return {
        'simbolo': simbolo,
        'registros': len(registros),
        'primer_valor': primer_valor,
        'ultimo_valor': ultimo_valor,
        'variacion': variacion,
        'variacion_porcentual': variacion_porcentual,
        **stats
    }

def mostrar_resumen_general(datos):
    """
    Muestra un resumen general de todos los datos.
    """
    empresas_unicas = set(d['simbolo'] for d in datos)

    print("=" * 70)
    print("           ANÁLISIS DE DATOS DE ACCIONES")
    print("=" * 70)
    print(f"\n📊 RESUMEN GENERAL:")
    print(f"   • Total de registros: {len(datos)}")
    print(f"   • Empresas analizadas: {len(empresas_unicas)}")
    print(f"   • Símbolos: {', '.join(sorted(empresas_unicas))}")

def mostrar_analisis_por_empresa(empresas_data):
    """
    Muestra el análisis detallado por empresa.
    """
    print(f"\n📈 ANÁLISIS POR EMPRESA:")
    print("-" * 70)

    analisis_empresas = []

    for simbolo in sorted(empresas_data.keys()):
        registros = empresas_data[simbolo]
        analisis = analizar_empresa(simbolo, registros)
        analisis_empresas.append(analisis)

        print(f"\n🏢 {simbolo}:")
        print(f"   • Registros: {analisis['registros']}")
        print(f"   • Rango: ${analisis['minimo']:.2f} - ${analisis['maximo']:.2f}")
        print(f"   • Promedio: ${analisis['promedio']:.2f}")
        print(f"   • Mediana: ${analisis['mediana']:.2f}")

        if analisis['registros'] > 1:
            print(f"   • Valor inicial: ${analisis['primer_valor']:.2f}")
            print(f"   • Valor final: ${analisis['ultimo_valor']:.2f}")
            print(f"   • Variación: ${analisis['variacion']:.2f} ({analisis['variacion_porcentual']:.2f}%)")
            print(f"   • Desviación estándar: ${analisis['desviacion']:.2f}")

            # Tendencia
            if analisis['variacion_porcentual'] > 0:
                print(f"   • Tendencia: 📈 ALZA")
            elif analisis['variacion_porcentual'] < 0:
                print(f"   • Tendencia: 📉 BAJA")
            else:
                print(f"   • Tendencia: ➡️ ESTABLE")

    return analisis_empresas

def mostrar_rankings(analisis_empresas):
    """
    Muestra rankings de las empresas por diferentes criterios.
    """
    if len(analisis_empresas) < 2:
        return

    print(f"\n🏆 RANKINGS:")
    print("-" * 50)

    # Por mayor valor máximo
    ranking_valor = sorted(analisis_empresas, key=lambda x: x['maximo'], reverse=True)
    print(f"\n📊 Por mayor valor registrado:")
    for i, empresa in enumerate(ranking_valor, 1):
        print(f"   {i}. {empresa['simbolo']}: ${empresa['maximo']:.2f}")

    # Por mayor variación porcentual
    empresas_con_variacion = [e for e in analisis_empresas if e['registros'] > 1]
    if empresas_con_variacion:
        ranking_variacion = sorted(empresas_con_variacion, key=lambda x: x['variacion_porcentual'], reverse=True)
        print(f"\n📈 Por mayor variación porcentual:")
        for i, empresa in enumerate(ranking_variacion, 1):
            print(f"   {i}. {empresa['simbolo']}: {empresa['variacion_porcentual']:.2f}%")

    # Por mayor volatilidad
    empresas_volatiles = [e for e in analisis_empresas if e['desviacion'] > 0]
    if empresas_volatiles:
        ranking_volatilidad = sorted(empresas_volatiles, key=lambda x: x['desviacion'], reverse=True)
        print(f"\n📊 Por mayor volatilidad:")
        for i, empresa in enumerate(ranking_volatilidad, 1):
            print(f"   {i}. {empresa['simbolo']}: ${empresa['desviacion']:.2f}")

def mostrar_datos_cronologicos(datos):
    """
    Muestra todos los datos ordenados cronológicamente.
    """
    datos_ordenados = sorted(datos, key=lambda x: (x['simbolo'], x['fecha']))

    print(f"\n📅 DATOS CRONOLÓGICOS DETALLADOS:")
    print("-" * 70)
    print(f"{'Fecha':<20} | {'Símbolo':<6} | {'Valor':<10}")
    print("-" * 70)

    for dato in datos_ordenados:
        print(f"{dato['fecha_str']:<20} | {dato['simbolo']:<6} | ${dato['valor']:>8.2f}")

def main():
    """
    Función principal que ejecuta todo el análisis.
    """
    # Ruta del archivo a procesar (corregido el nombre)
    ruta_archivo = '/content/drive/MyDrive/ClassFiles/informe_acciones.txt'

    print(f"Procesando archivo: {ruta_archivo}")

    # Extraer datos
    datos = extraer_datos_acciones(ruta_archivo)

    if not datos:
        print("❌ No se pudieron extraer datos válidos del archivo.")
        print("Verifica que el archivo existe y tiene el formato correcto.")
        return

    # Agrupar por empresa
    empresas_data = agrupar_por_empresa(datos)

    # Mostrar análisis completo
    mostrar_resumen_general(datos)
    analisis_empresas = mostrar_analisis_por_empresa(empresas_data)
    mostrar_rankings(analisis_empresas)
    mostrar_datos_cronologicos(datos)

    print(f"\n✅ Análisis completado exitosamente!")

if __name__ == "__main__":
    main()

Streaming output truncated to the last 5000 lines.
2024-09-02 12:34:26  | MSFT   | $ 1361.55
2024-09-02 12:34:26  | MSFT   | $  747.46
2024-09-02 12:34:26  | MSFT   | $  146.78
2024-09-02 12:34:26  | MSFT   | $ 1155.27
2024-09-02 12:34:26  | MSFT   | $  974.52
2024-09-02 12:34:26  | MSFT   | $  862.98
2024-09-02 12:34:26  | MSFT   | $ 1131.76
2024-09-02 12:34:26  | MSFT   | $  327.10
2024-09-02 12:34:26  | MSFT   | $  776.10
2024-09-02 12:34:26  | MSFT   | $  233.57
2024-09-02 12:34:26  | MSFT   | $  503.03
2024-09-02 12:34:26  | MSFT   | $  130.05
2024-09-02 12:34:26  | MSFT   | $  741.02
2024-09-02 12:34:26  | MSFT   | $ 1282.71
2024-09-02 12:34:26  | MSFT   | $  869.88
2024-09-02 12:34:26  | MSFT   | $  104.05
2024-09-02 12:34:26  | MSFT   | $  673.34
2024-09-02 12:34:26  | MSFT   | $ 1049.81
2024-09-02 12:34:26  | MSFT   | $  491.68
2024-09-02 12:34:26  | MSFT   | $  766.22
2024-09-02 12:34:26  | MSFT   | $  557.37
2024-09-02 12:34:26  | MSFT   | $  548.46
2024-09-02 12:34:26  | MS